## Irreversible CPE Model Example

### Imports

In [1]:
import os
import numpy as np
from polypesto.core.params import ParameterGroup
from polypesto.core.study import Study, create_study
from polypesto.core.experiment import create_simulation_conditions
from polypesto.models.CRP2 import IrreversibleCPE
from polypesto.utils.paths import setup_data_dirs
from polypesto.visualization import (
    plot_all_results,
    plot_all_ensemble_predictions,
    plot_all_comparisons_1D,
)

%load_ext autoreload
%autoreload 2

in apply patches
before from_yaml patch
after from_yaml patch


### Define Conditions and Parameters

In [2]:
notebook_path = os.getcwd() 
DATA_DIR, TEST_DIR = setup_data_dirs(notebook_path)

simulation_params = ParameterGroup.create_parameter_grid(
    {
        "rA": [0.1],    # Reactivity ratios for monomer A
        "rB": [0.1],    # Reactivity ratios for monomer B
    },
    filter_fn=lambda p: p["rA"] >= p["rB"],
)

In [3]:
# Define fitting parameters

fit_params = IrreversibleCPE.get_default_parameters()
obs_df = IrreversibleCPE.create_observables(
    observables={"fA": "fA"}, noise_value=0.02  # Specify observables and noise value
)

In [4]:
# Define experimental configurations

t_eval = np.arange(0, 1, 0.05)
fA0s = [[0.1]]
cM0s = [[1.0]]
names = [f"fA0_{fA0}_cM0_{cM0}" for fA0, cM0 in zip(fA0s, cM0s)]
ntrials = len(fA0s)
assert len(fA0s) == len(cM0s), "fA0s and cM0s must have the same length"

In [5]:
# Establish simulation conditions

conditions = create_simulation_conditions(
    dict(
        name=names,
        t_eval=[t_eval] * ntrials,
        conditions=dict(fA0=fA0s, cM0=cM0s),
        fit_params=[fit_params] * ntrials,
        noise_level=[0.02] * ntrials,
    )
)

### Create Study

In [6]:
study = create_study(
    model=IrreversibleCPE,
    simulation_params=simulation_params,
    conditions=conditions,
    obs_df=obs_df,
    base_dir=DATA_DIR,
    overwrite=True,
)

/Users/shivanimadhan/Documents/research/parameter_estimation/PolyPESTO/.venv/lib/python3.13/site-packages/petab/v2/__init__.py:12: UserWarning: Support for PEtab2.0 and all of petab.v2 is experimental and subject to changes!
  warn(
Visualization table not available. Skipping.
Visualization table not available. Skipping.
Visualization table not available. Skipping.


Creating new study.
Creating SBML model: irreversible_cpe
Creating irreversible CRP parameters.
                 filename : /Users/shivanimadhan/Documents/research/parameter_estimation/PolyPESTO/polypesto/data/examples/fA0_[0.1]_cM0_[1.0]/petab/common/irreversible_cpe.xml
         file size (byte) : 19026
           read time (ms) : 1.070976
        c-check time (ms) : 0.396729
      validation error(s) : 0
    (consistency error(s)): 0
    validation warning(s) : 0
  (consistency warning(s)): 0


### Run Parameter Estimation

In [7]:
study.run_parameter_estimation(
    config=dict(
        optimize=dict(n_starts=50, method="Nelder-Mead"),
        profile=dict(method="Nelder-Mead"),
        sample=dict(n_samples=10000, n_chains=3),
    ),
    overwrite=False,
)

Engine will use up to 10 processes (= CPU count).


Running parameter estimation for all experiments...
Running parameter estimation for fA0_[0.1]_cM0_[1.0], p_000...
	Running optimize_problem with {'n_starts': 50, 'method': 'Nelder-Mead'}


100%|██████████| 50/50 [00:04<00:00, 11.23it/s]


	Running profile_problem with {'method': 'Nelder-Mead'}


  0%|          | 0/2 [00:00<?, ?it/s]Next guess for rA in direction -1 is -1.0366. Step size: -0.1000.
Optimization successful for rA=-1.0366. Start fval -59.840185, end fval -59.842554.
Next guess for rA in direction -1 is -1.0559. Step size: -0.0193.
Optimization successful for rA=-1.0559. Start fval -59.841939, end fval -59.841940.
Next guess for rA in direction -1 is -1.0785. Step size: -0.0226.
Optimization successful for rA=-1.0785. Start fval -59.841020, end fval -59.841020.
Next guess for rA in direction -1 is -1.1046. Step size: -0.0261.
Optimization successful for rA=-1.1046. Start fval -59.839639, end fval -59.839643.
Next guess for rA in direction -1 is -1.1346. Step size: -0.0300.
Optimization successful for rA=-1.1346. Start fval -59.837573, end fval -59.837574.
Next guess for rA in direction -1 is -1.1685. Step size: -0.0340.
Optimization successful for rA=-1.1685. Start fval -59.834475, end fval -59.834481.
Next guess for rA in direction -1 is -1.2068. Step size: -0.038

	Running sample_problem with {'n_samples': 10000, 'n_chains': 3}


100%|██████████| 10000/10000 [00:20<00:00, 492.45it/s]
Elapsed time: 20.201431


	Saving results to /Users/shivanimadhan/Documents/research/parameter_estimation/PolyPESTO/polypesto/data/examples/fA0_[0.1]_cM0_[1.0]/pypesto/p_000/results.hdf5
Done running parameter estimation.
Plotting results...
Plotting all comparisons...
Plotting 1 comparisons...
Plotting comparison for parameter set 1/1
Filtering experiments...
Filter condition ID: None
Filter parameter ID: p_000
Filtering experiments...
Filter condition ID: None
Filter parameter ID: p_000


In [8]:
plot_all_comparisons_1D(study)
plot_all_results(study)

Plotting 1 comparisons...
Plotting comparison for parameter set 1/1
Filtering experiments...
Filter condition ID: None
Filter parameter ID: p_000
Found 1 experiments for parameter set p_000
Filtering experiments...
Filter condition ID: None
Filter parameter ID: p_000
['rA', 'rB']
['rA', 'rB']
Plotting comparison for condition fA0_[0.1]_cM0_[1.0], parameter set p_000
{'rA': {0.95: (np.float64(0.005195775991453491), np.float64(0.3104701368234285), np.float64(40.79708826835601))}, 'rB': {0.95: (np.float64(0.0015156227615562795), np.float64(0.10163960996331543), np.float64(0.4374582422393346))}}
Parameter rA confidence intervals: 1.7763694142472695, -0.5079801669415758, 2.1186093350777595
Parameter rB confidence intervals: 1.826471870931734, -0.9929370102008188, 0.6338736137750229


/Users/shivanimadhan/Documents/research/parameter_estimation/PolyPESTO/.venv/lib/python3.13/site-packages/pypesto/visualize/sampling.py:1223: UserWarning: Burn in index not found in the results, the full chain will be shown.
You may want to use, e.g., `pypesto.sample.geweke_test`.
  nr_params, params_fval, theta_lb, theta_ub, _ = get_data_to_plot(
/Users/shivanimadhan/Documents/research/parameter_estimation/PolyPESTO/.venv/lib/python3.13/site-packages/scipy/_lib/_util.py:1279: RuntimeWarning: divide by zero encountered in vecdot
  return np.vecdot(x1, x2, axis=axis)
/Users/shivanimadhan/Documents/research/parameter_estimation/PolyPESTO/.venv/lib/python3.13/site-packages/scipy/_lib/_util.py:1279: RuntimeWarning: overflow encountered in vecdot
  return np.vecdot(x1, x2, axis=axis)
/Users/shivanimadhan/Documents/research/parameter_estimation/PolyPESTO/.venv/lib/python3.13/site-packages/scipy/_lib/_util.py:1279: RuntimeWarning: invalid value encountered in vecdot
  return np.vecdot(x1, x2,